# Built-In BlackBox Attacks

Notebook demonstrating built-in BlackBox attacks (NES, Finite Difference, SPSA, RGF) from
`adversarial_lab.arsenal.adversarial.blackbox`.

**Notes**
- This notebook imports `*` from `adversarial_lab.arsenal.adversarial.blackbox` as requested.
- Replace the **MODEL SETUP** cell with your real model and preprocessing function.


In [ ]:
from adversarial_lab.arsenal.adversarial.blackbox import *

from adversarial_lab.utils import Plotting

from PIL import Image
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

In [ ]:
MODEL = "mnist_digits"
TARGET_CLASS = 2

if MODEL == "mnist_digits":
    DIGIT = 1
    IMAGE = 1
    MODEL_PATH = f"models/{MODEL}.h5"
    IMAGE_PATH = f"data/digits/{DIGIT}/{IMAGE}.png"

In [ ]:
model = tf.keras.models.load_model(MODEL_PATH)

def pred_fn(samples, *args, **kwargs):
    samples = np.array(samples)
    preds = model.predict(samples, verbose=0)
    return [pred for pred in preds]

In [ ]:
image = Image.open(IMAGE_PATH)
image_array = np.array(image)

predictions = pred_fn([image_array])
print(f"Predicted class: {np.argmax(predictions)}")
print(f"True class: {DIGIT}")
print(f"Predicticted Class Probability: {np.max(predictions)}")

In [ ]:
fd = FiniteDifferenceAttack(pred_fn=pred_fn, epsilon=1, max_queries=5000, batch_size=32, block_size=9)
adv_fd, noise_meta = fd.attack(sample=image_array, target_class=TARGET_CLASS, epochs=50)

Plotting.plot_images_and_noise(image_array, adv_fd)

In [ ]:
nes = NESAttack(pred_fn=pred_fn, sigma=0.7, samples=50, learning_rate=1.0, verbose=2)
adv_nes, noise_meta = nes.attack(sample=image_array, target_class=TARGET_CLASS, epochs=50)

Plotting.plot_images_and_noise(image_array, adv_nes)

In [ ]:
nes = SPSAAttack(pred_fn=pred_fn, epsilon=20, samples=50, learning_rate=1.0, verbose=2)
adv_spsa, noise_meta = nes.attack(sample=image_array, target_class=TARGET_CLASS, epochs=50)

Plotting.plot_images_and_noise(image_array, adv_spsa)

In [ ]:
nes = RGFAttack(pred_fn=pred_fn, epsilon=55, samples=50, learning_rate=1.0, verbose=2)
adv_rgf, noise_meta = nes.attack(sample=image_array, target_class=TARGET_CLASS, epochs=50)

Plotting.plot_images_and_noise(image_array, adv_rgf)